In [1]:
import ccf2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import warnings
import lightgbm as lgb
import xgboost
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split,cross_validate,TimeSeriesSplit
from sklearn.metrics import mean_squared_error as mse
from sklearn.grid_search import GridSearchCV
warnings.filterwarnings('ignore')

path = '../input/'

D:\programfiles\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\programfiles\anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# load data

In [2]:
data = ccf2.loaddata(path)

In [5]:
data.head()

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,mt,log_label,log_popularity
0,310000,0,0,0,上海,1,2016,1479.0,11.0,106.0,292.0,1,5.676754,7.299121
1,530000,0,0,0,云南,1,2016,1594.0,11.0,106.0,466.0,1,6.144186,7.374002
2,150000,0,0,0,内蒙古,1,2016,1479.0,11.0,106.0,257.0,1,5.549076,7.299121
3,110000,0,0,0,北京,1,2016,2370.0,11.0,106.0,408.0,1,6.011267,7.770645
4,510000,0,0,0,四川,1,2016,3562.0,11.0,106.0,610.0,1,6.413459,8.178077


In [6]:
data.describe()

,adcode,bodyType,id,model,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,mt,log_label,log_popularity
count,36960.000000,36960.000000,36960.000000,36960.000000,36960.000000,36960.000000,31680.000000,31680.000000,31680.000000,31680.000000,36960.000000,31680.000000,31680.000000
mean,347727.272727,0.766667,382.452381,29.500000,5.928571,2016.714286,2311.965941,250.479167,1989.809722,608.709122,14.500000,5.880215,7.240776
std,136281.363156,0.803476,1104.799221,17.318337,3.514595,0.699864,2980.547086,318.648963,2324.517694,779.076734,8.077856,1.059051,1.025449
min,110000.000000,0.000000,0.000000,0.000000,1.000000,2016.000000,25.000000,0.000000,0.000000,2.000000,1.000000,0.693147,3.218876
25%,230000.000000,0.000000,0.000000,14.750000,3.000000,2016.000000,700.000000,54.000000,335.000000,182.000000,7.750000,5.204007,6.551080
50%,355000.000000,1.000000,0.000000,29.500000,5.500000,2017.000000,1445.500000,149.500000,1432.500000,363.000000,14.500000,5.894403,7.276211
75%,440000.000000,1.000000,0.000000,44.250000,9.000000,2017.000000,2816.000000,307.000000,2712.000000,737.000000,21.250000,6.602588,7.943073
max,610000.000000,3.000000,5368.000000,59.000000,12.000000,2018.000000,116300.000000,2834.000000,20770.000000,15317.000000,28.000000,9.636719,11.663928


In [3]:
import math
# data['model_weight'] = data.groupby('model')['label'].transform('mean') 
# data['n_label'] = data['label'] / data['model_weight'] 
data['log_label'] = data['label'].apply(lambda x: math.log(x))
data['log_popularity'] = data['popularity'].apply(lambda x: math.log(x))
# data['log_carCommentVolum'] = data['carCommentVolum'].apply(lambda x: math.log(x+1))
# data['log_newsReplyVolum'] = data['newsReplyVolum'].apply(lambda x: math.log(x+1))

# feature

## 构建特征

### 平移12特征

In [4]:
data,shift_feat = ccf2.genShitFeat(data,[12])

### 分组特征

In [5]:
# target=['popularity','carCommentVolum','newsReplyVolum','label']
target=['label']
fea_list = ['adcode','bodyType','model','regMonth']
data,stat_feat = ccf2.genStatFeat(data,fea_list,target)

统计列的sum,mean,max,min,分位数0.2,分位数0.5,分位数0.8
构造特征: adcode
构造特征: adcode _ label
构造特征: bodyType
构造特征: bodyType _ label
构造特征: model
构造特征: model _ label
构造特征: regMonth
构造特征: regMonth _ label


In [6]:
data.tail()

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,...,label_model_median2,label_model_median5,label_model_median8,label_regMonth_sum,label_regMonth_mean,label_regMonth_max,label_regMonth_mim,label_regMonth_median2,label_regMonth_median5,label_regMonth_median8
36955,350000,0,5364,59,福建,4,2018,NaN,NaN,NaN,...,82.4,138.0,262.0,1348827.0,510.919318,8733.0,11.0,137.0,316.0,722.2
36956,210000,0,5365,59,辽宁,4,2018,NaN,NaN,NaN,...,82.4,138.0,262.0,1348827.0,510.919318,8733.0,11.0,137.0,316.0,722.2
36957,500000,0,5366,59,重庆,4,2018,NaN,NaN,NaN,...,82.4,138.0,262.0,1348827.0,510.919318,8733.0,11.0,137.0,316.0,722.2
36958,610000,0,5367,59,陕西,4,2018,NaN,NaN,NaN,...,82.4,138.0,262.0,1348827.0,510.919318,8733.0,11.0,137.0,316.0,722.2
36959,230000,0,5368,59,黑龙江,4,2018,NaN,NaN,NaN,...,82.4,138.0,262.0,1348827.0,510.919318,8733.0,11.0,137.0,316.0,722.2


In [7]:
cate_feat = ['adcode', 'bodyType', 'model', 'regMonth']
num_feat = ['regYear'] + shift_feat
features = num_feat+stat_feat

In [8]:
features = list(set(features))

# 构建模型

## 拆分数据

In [10]:
train_idx = (data['mt'] <= 24)
test_idx = (data['mt'] > 24)

trainSet = data.loc[train_idx]
testSet = data.loc[test_idx]
# train_x = data[train_idx][features]
# train_y = data[train_idx]['n_label']
# train_y2 = data[train_idx]['label']
# valid_idx = (data['mt'].between(21, 24))
# valid_x = data[valid_idx][features]
# valid_y = data[valid_idx]['n_label']

## cv 交叉验证

In [12]:
lgb_model = lgb.LGBMRegressor(
        num_leaves=32, reg_alpha=1, reg_lambda=0.1, objective='mse',
        max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=np.random.randint(1000),
        n_estimators=5000, subsample=0.8, colsample_bytree=0.8,
    )

In [35]:
# from sklearn.utils.multiclass import type_of_target
# type_of_target(train_y)

'continuous'

In [14]:
# kfd = KFold(n_splits=5,random_state=2020)
tscv = TimeSeriesSplit(n_splits=4)
# res_prob = pd.DataFrame()
res = pd.DataFrame()

index = 1
for train_idx,test_idx in tscv.split(trainSet):
    print('-------------------','Kflod:',index,'------------------')
    
    lgb_model.fit(trainSet.loc[train_idx][features],trainSet.loc[train_idx]['log_label'], 
                  eval_set=[(trainSet.loc[train_idx][features],trainSet.loc[train_idx]['log_label']),
                            (trainSet.loc[test_idx][features],trainSet.loc[test_idx]['log_label'])],
                  categorical_feature=cate_feat, 
                  early_stopping_rounds=500,
                  verbose=100)
#     x_pred = lgb_model.predict(train_x[test_idx], num_iteration=lgb_model.best_iteration_)
    test_pred = lgb_model.predict(testSet[features], num_iteration=lgb_model.best_iteration_)
    res['prob_%s' % str(index)] = test_pred
    index = index+1

------------------- Kflod: 1 ------------------
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's l2: 0.113615	valid_1's l2: 0.178107
[200]	valid_0's l2: 0.0521486	valid_1's l2: 0.124155
[300]	valid_0's l2: 0.0289683	valid_1's l2: 0.104323
[400]	valid_0's l2: 0.0219356	valid_1's l2: 0.0990136
[500]	valid_0's l2: 0.0187091	valid_1's l2: 0.0964728
[600]	valid_0's l2: 0.0186405	valid_1's l2: 0.0964144
[700]	valid_0's l2: 0.0186405	valid_1's l2: 0.0964145
[800]	valid_0's l2: 0.0186405	valid_1's l2: 0.0964145
[900]	valid_0's l2: 0.0186405	valid_1's l2: 0.0964145
[1000]	valid_0's l2: 0.0186405	valid_1's l2: 0.0964145
Early stopping, best iteration is:
[504]	valid_0's l2: 0.0186406	valid_1's l2: 0.0964087
------------------- Kflod: 2 ------------------
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's l2: 0.116247	valid_1's l2: 0.273888
[200]	valid_0's l2: 0.0564047	valid_1's l2: 0.221645
[300]	valid_0's l2: 0.0331893	valid_1's l2: 0.2

In [15]:
res.head()

,prob_1,prob_2,prob_3,prob_4
0,5.758685,5.791195,5.586931,5.387570
1,5.897129,6.017053,6.106881,5.899896
2,5.579192,5.534088,5.325514,5.183426
3,6.094716,6.186754,5.879524,5.949331
4,6.252155,6.325909,6.093264,6.043794


In [16]:
sub = ccf2.genLogSub(data,res,4)

In [17]:
sub.to_csv(path+'sub/sub_cv5.csv',index=False,header=True)

In [20]:
df_imp = ccf2.featImport(lgb_model,features)

In [21]:
df_imp

,features,import
19,model,2318
59,adcode,1376
89,shift_model_adcode_mt_label_12,919
5,label_regMonth_median2,311
3,label_adcode_median5,259
4,newsReplyVolum_model_median5,195
114,regMonth,142
33,popularity_adcode_mim,139
13,popularity_adcode_median2,126
84,label_model_max,120
